<a href="https://www.kaggle.com/code/kavya2099/digit-recognizer?scriptVersionId=175991551" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reading training and testing dataset

In [ ]:
train= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train.shape
train.head()

In [ ]:
test= pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test.shape
test.head()

dropping label column from x_train dataset and adding it in y_train 

In [ ]:
y_train= train['label']
y_train

In [ ]:
x_train= train.drop(labels='label',axis=1)
x_train

Visualizing the count of numbers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
count= sns.countplot(x=y_train)
y_train.value_counts()

To see a sample image

In [ ]:
image=x_train.iloc[0]
# Convert the dataset to a 2D numpy array
image_array = image.to_numpy()

# Reshape the 1D array to a 2D image (e.g., 5x5 pixels)
image_shape = (28, 28) #this is based on 784 values which is from 28x28
image = image_array.reshape(image_shape)

# Display the image
plt.imshow(image, cmap='gray') 
#The cmap parameter specifies the color map to be used for displaying the image. In this case, 'gray' indicates that the image should be displayed in grayscale (black and white).
plt.axis('off')  # Hide axes
plt.show()


In [ ]:
image=x_train.iloc[78]
# Convert the dataset to a 2D numpy array
image_array = image.to_numpy()

# Reshape the 1D array to a 2D image (e.g., 5x5 pixels)
image_shape = (28, 28) #this is based on 784 values which is from 28x28
image = image_array.reshape(image_shape)

# Display the image
plt.imshow(image, cmap='gray') 
#The cmap parameter specifies the color map to be used for displaying the image. In this case, 'gray' indicates that the image should be displayed in grayscale (black and white).
plt.axis('off')  # Hide axes
plt.show()


# Normalization, Reshape and Label Encoding
## Normalization:
**Purpose**: Normalization ensures that pixel values are within a consistent range, typically between 0 and 1 or -1 and 1.

**Reasons**:
* **Stabilizes Training**: Normalizing pixel values prevents large variations in input data, which can lead to unstable training. It helps the model converge faster and more reliably.
* **Gradient Descent**: Gradient descent optimization algorithms work better when features are on a similar scale.
* **Activation Functions**: Some activation functions (e.g., sigmoid, tanh) perform better with normalized inputs.
* **Example**: Divide pixel values by 255 to scale them between 0 and 1.

## Reshaping:
**Purpose**: Images are typically represented as 2D arrays (height x width x channels). CNNs expect a specific input shape.

**Reasons**:
* **Input Shape**: CNNs require a consistent input shape (e.g., (height, width, channels)).
* **Flattening**: If using fully connected layers after CNN layers, reshaping is necessary to flatten the 2D feature maps into a 1D vector.
* **Example**: Reshape a grayscale image (28x28 pixels) to (28, 28, 1) or an RGB image (28x28x3) to (28, 28, 3).

## Label Encoding:
**Purpose**: Convert categorical labels (classes) into numerical representations.

**Reasons**:
* **Model Input**: CNNs require numerical labels for training.
* **Loss Calculation**: Loss functions (e.g., cross-entropy) compare predicted class probabilities with true labels.
* **Example**: Encode class labels (e.g., “cat,” “dog,” “bird”) as integers (e.g., 0, 1, 2).

2 => [0,0,1,0,0,0,0,0,0,0]

4 => [0,0,0,0,1,0,0,0,0,0]

### Can We Proceed Without These Steps?

* Technically, you can train a CNN without normalization, reshaping, or label encoding, but it’s not recommended.
* Skipping normalization may lead to slow convergence or poor performance.
* Skipping reshaping will cause shape mismatches between input data and model architecture.
* Skipping label encoding will result in incorrect loss calculations and misinterpretation of class probabilities.



In [ ]:
#normalize

x_train=x_train/255.0 
test=test/255.0 

In [ ]:
#reshape

x_train=x_train.to_numpy()
test=test.to_numpy()

x_train=x_train.reshape(-1,28,28,1)
#(batch_size, height, width, channels)
#The batch size affects how gradients are computed during backpropagation. It impacts the stability and speed of training.
test=test.reshape(-1,28,28,1)

print("x_train shape: ",x_train.shape)
print("test shape: ",test.shape)

In [ ]:
#encoding
import keras
from keras.utils import to_categorical  # convert to one-hot-encoding
y_train = to_categorical(y_train, num_classes = 10)

# Test train split
Since we have aready have split data, the reason we split up here is to use it for validating.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2) #splitting 80-20

print("x_train shape",x_train.shape)
print("x_test shape",x_val.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_val.shape)

# Training and evaluation

* Create models with layers
* Compile model
* Fit model
* Evaluate the model

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

### Creating CNN model with layers

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),padding='Same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3),padding='Same',activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))
#fully connected layer
model.add(Flatten())
model.add(Dense(128,'relu'))
model.add(Dropout(0.2))
model.add(Dense(10,'softmax'))


Initializing the optimizer

In [ ]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

**learning_rate** specifies the learning rate for the optimizer, **beta_1** and **beta_2** are the exponential decay rates for the moment estimates, and **epsilon** is a small value added to the denominator for numerical stability. 

These values are the defaults used in TensorFlow's implementation of Adam, but you can adjust them based on your specific needs and the characteristics of your model and dataset.

### Compiling the model 

In [ ]:
# Compile the model
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

#categorical_crossentropy--->It generalizes binary cross-entropy to multiple classes. 
#The model predicts class probabilities for each category, and the loss is minimized based on the true class probabilities 1.

The **metrics** argument expects a **list of strings**, even if you’re specifying only one metric. So, always use the list format to ensure proper functionality. 

Say you have a dataset of 10 examples (or samples). You have a batch size of 2, and you've specified you want the algorithm to run for 3 epochs. Therefore, in each epoch, you have 5 batches (10/2 = 5). Each batch gets passed through the algorithm, therefore you have 5 iterations per epoch.

In [ ]:
epochs=10
batch_size=250

## Data Augumentation

 Data augmentation supplements the creation of data variations that can help a model improve the accuracy of its predictions.
 
* To avoid overfitting problem, we need to expand artificially our handwritten digit dataset
* Alter the training data with small transformations to reproduce the variations of digit.
* For example, the number is not centered The scale is not the same (some who write with big/small numbers) The image is rotated.

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

### Fitting the model



In [ ]:
# Fit the model

train_iterator = datagen.flow(x_train, y_train, batch_size=batch_size)

"""This line creates an iterator for data augmentation using the ImageDataGenerator (datagen).
It generates augmented batches of training data.
The x_train and y_train are the input features (images) and corresponding labels (target values) for your training dataset.
The batch_size determines how many samples are processed at once in each batch."""


# Train your model
history = model.fit(train_iterator,epochs = epochs, validation_data = (x_val,y_val), steps_per_epoch=len(x_train) // batch_size)

Converting the results saved as dataset


The **keras.callbacks.History** (which we saved here as **history**)object returned by the model.fit() method contains information about the training process, including metrics such as loss and accuracy recorded at each epoch. To access the contents inside the History object, you can use the following methods:

**Accessing Training Metrics:**
The **History** object contains a dictionary called **history** that holds the training metrics.

In [ ]:
history_df= pd.DataFrame(history.history)
history_df

Plotting the **Training and Validation accuracy and loss**

In [ ]:


print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_accuracy'].max()))


plt.figure(figsize=(8, 6))
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


plt.figure(figsize=(8, 6))
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()




### Testing with new samples

In [ ]:
import cv2
# Load the sample image

for i in range(1, 6):  # Assuming you have images named test1.jpg, test2.jpg, ..., test5.jpg
    # Construct the filename for the current image
    image_path = f'/kaggle/input/sample-test-set/test {i}.jpg'
    test_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Replace 'sample_image.jpg' with your image file path


    # Preview sample image
    plt.imshow(test_image, cmap='gray')

    # Format Image
    img_resized = cv2.resize(test_image, (28, 28), interpolation=cv2.INTER_LINEAR)
    img_resized = cv2.bitwise_not(img_resized)

    # Preview reformatted image
    plt.imshow(img_resized, cmap='gray')
    plt.show()

    # Prepare the image for prediction
    input_image = img_resized.reshape(1, 28, 28, 1)  # Reshape to match the input format of your model
    input_image = input_image.astype('float32') / 255  # Normalize the pixel values

    prediction = model.predict(input_image)
    predicted_digit = np.argmax(prediction)

    print("Predicted Digit:", predicted_digit)

Out of 5 test samples, it was able to identify only 2 correctly

### References

https://www.kaggle.com/code/kanncaa1/convolutional-neural-network-cnn-tutorial/notebook